# 02 — Auto-Embedding

MongoDB Community Edition 8.2+ includes **native auto-embedding** in the vector search engine (`mongot`).

How it works:
1. You declare `"type": "autoEmbed"` on a text field in your index definition, choosing a VoyageAI model.
2. MongoDB generates and keeps embeddings in sync automatically — for existing docs, new inserts, and updates.
3. At query time, use `query: { text: '...' }` in `$vectorSearch` — MongoDB embeds your query too.

**You write zero embedding code.** No API calls, no vector fields, no backfill scripts.

> **Before you start:** This notebook requires your personal VoyageAI API key to be set as a GitHub Codespace secret named `VOYAGE_API_KEY`.
> Go to **github.com → Settings → Codespaces → Secrets** and add your key there — then (re)open this Codespace.
> Each student uses their own key. The key is forwarded automatically to `mongot`, which handles all embedding calls on your behalf.

**Steps:**
1. Create an `autoEmbed` index on `listings.description`
2. Insert documents normally — no vectors, no embedding step
3. Query with `query.text` — MongoDB embeds and searches for you
4. Pre-filtered text search
5. Cross-model query: index with `voyage-4`, query with `voyage-4-lite`

In [ ]:
// ── Setup ────────────────────────────────────────────────────────────────────
import { MongoClient } from 'mongodb';

const INDEX_NAME = 'auto_embed_index';

const client = new MongoClient(process.env.MONGODB_URI!);
await client.connect();
const db  = client.db('voyage_lab');
const col = db.collection<{ _id: string; [key: string]: unknown }>('listings');

console.log('Connected. Listings:', await col.countDocuments());

## Step 1 — Create an `autoEmbed` vector search index

The only configuration needed: which field to embed (`description`) and which model to use (`voyage-4`).  
MongoDB handles everything else.

In [ ]:
// ── Create autoEmbed index ────────────────────────────────────────────────────
try {
  await col.dropSearchIndex(INDEX_NAME);
  await new Promise(r => setTimeout(r, 2000));
} catch { /* didn't exist */ }

await col.createSearchIndex({
  name: INDEX_NAME,
  type: 'vectorSearch',
  definition: {
    fields: [
      {
        type:     'autoEmbed',   // ← tells MongoDB to generate embeddings
        modality: 'text',
        path:     'description', // ← field to embed
        model:    'voyage-4',    // ← VoyageAI model
      },
      { type: 'filter', path: 'price' },
      { type: 'filter', path: 'property_type' },
      { type: 'filter', path: 'bedrooms' },
    ],
  },
});

console.log('Index creation requested. Waiting for READY status...');
for (let i = 0; i < 30; i++) {
  await new Promise(r => setTimeout(r, 5000));
  const cursor = col.listSearchIndexes(INDEX_NAME);
  for await (const idx of cursor) {
    console.log(` status: ${idx.status}`);
    if (idx.queryable) { console.log('Index is ready.'); break; }
  }
  const [idx] = await col.listSearchIndexes(INDEX_NAME).toArray();
  if (idx?.queryable) break;
}

## Step 2 — Insert documents with no vector field

Regular `insertMany` — just plain documents. MongoDB auto-embeds the `description` field in the background.

In [ ]:
// ── Insert new listings — no embedding code ──────────────────────────────────
const newListings = [
  { _id: 'ae-001', name: 'Beachfront studio in Malibu',            description: 'Wake up to ocean waves in this light-filled studio steps from the Pacific. Perfect for surfers and sunset chasers.', price: 320, bedrooms: 1, property_type: 'Studio'    },
  { _id: 'ae-002', name: 'Mountain cabin in Aspen',                description: 'Secluded log cabin with hot tub, fireplace and ski-in/ski-out access. Stunning alpine views.',                        price: 550, bedrooms: 3, property_type: 'Cottage'   },
  { _id: 'ae-003', name: 'Urban loft in Brooklyn',                 description: 'Industrial-chic loft in a converted warehouse. High ceilings, exposed brick, and rooftop access.',                    price: 195, bedrooms: 1, property_type: 'Loft'      },
  { _id: 'ae-004', name: 'Vineyard cottage in Napa Valley',        description: 'Nestled among grapevines, this romantic cottage offers wine tastings, garden walks, and farm-to-table breakfasts.',   price: 430, bedrooms: 2, property_type: 'Cottage'   },
  { _id: 'ae-005', name: 'Historic apartment in Rome, Trastevere', description: 'Stay in a Renaissance palazzo. Frescoed ceilings, stone floors, and a terrace overlooking ancient cobblestones.',    price: 280, bedrooms: 2, property_type: 'Apartment' },
];

// Drop previous inserts if re-running
await col.deleteMany({ _id: { $in: newListings.map(l => l._id) } });
await col.insertMany(newListings);

// Verify: no embedding field on the documents — MongoDB handles it internally
const sample = await col.findOne({ _id: 'ae-001' }, { projection: { name: 1, embedding: 1 } });
console.log('Document (no embedding field):', JSON.stringify(sample));
// OBSERVE: No 'embedding' field — it lives in a separate system collection managed by mongot.

## Step 3 — Query with `query.text`

Use `query: { text: '...' }` in `$vectorSearch` instead of `queryVector`.  
MongoDB calls VoyageAI at query-time and runs the ANN search — all in one pipeline stage.

In [ ]:
// ── $vectorSearch with query.text — no manual embedding ──────────────────────
const results = await col.aggregate([
  {
    $vectorSearch: {
      index:         INDEX_NAME,
      path:          'description',
      query:         { text: 'romantic wine country getaway surrounded by nature' },
      model:         'voyage-4',
      numCandidates: 50,
      limit:         5,
    },
  },
  {
    $project: {
      name:          1,
      property_type: 1,
      price:         1,
      score:         { $meta: 'vectorSearchScore' },
    },
  },
]).toArray();

console.log('Results for: "romantic wine country getaway surrounded by nature"\n');
console.table(results.map(r => ({ name: r.name, price: r.price, score: (r.score as number).toFixed(4) })));
// OBSERVE: The Napa Valley cottage ranks highest — no embedding code written.

In [ ]:
// ── Cleanup ───────────────────────────────────────────────────────────────────
await col.deleteMany({ _id: { $in: ['ae-001', 'ae-002', 'ae-003', 'ae-004', 'ae-005'] } });
await client.close();
console.log('Done.');